Standard dp problem

In [569]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.matlib
%matplotlib inline
import quantecon as qe
import scipy.optimize as optimize
import scipy.sparse as sparse
from quantecon import compute_fixed_point
from quantecon.markov import DiscreteDP

In [41]:
# Define a random model I found on the internet
class model:

    def __init__(self, B=10, M=5, α=0.5, β=0.9):
        """
        R = reward array (n x m)
        Q = trnasition probability array (n x m x n)
        β = discount factor [0,1)
        α = quanticy to store and the action
        c = consume (s - α)
        M = global bound of storage
        u(c) = flow utility (c^α)
        s' = next periods stock (a + U)
        U = output, uniform distribution {0,...,B}
        B = maximum output
        S = state space {0,...,M+B}
        n = number of states (M + B + 1) since 0 to M + B
        A(s) = set of feasible actions {0,...,min{s,M}}
        r(s,α) = reward function (u(s - α))
        Q(s,α,s') = transition probability (equation 4.3)
        """

        self.B, self.M, self.α, self.β  = B, M, α, β
        self.n = B + M + 1
        self.m = M + 1

        self.R = np.empty((self.n, self.m))
        self.Q = np.zeros((self.n, self.m, self.n))

        self.populate_Q() 
        self.populate_R()

    def u(self, c):
        return c**self.α

    def populate_R(self):
        """
        Populate the R matrix, with R[s, a] = -np.inf for infeasible
        state-action pairs.
        """
        for s in range(self.n):
            for a in range(self.m):
                self.R[s, a] = self.u(s - a) if a <= s else -np.inf

    def populate_Q(self):
        """
        Populate the Q matrix by setting

            Q[s, a, s'] = 1 / (1 + B) if a <= s' <= a + B

        and zero otherwise.
        """

        for a in range(self.m):
            self.Q[:, a, a:(a + self.B + 1)] = 1.0 / (self.B + 1)

In [549]:
# Define the leapfrogging model
class lf:

    def __init__(self, Cmax = 5, Cmin = 0, nC = 4, pf = 1, k0 = 0, k1 = 8.3, k2 = 1, R = 0.05, Dt = 1):
        """
        Descibe the variables here :(
        all the standard fixed parameters
        """

        self.Cmax, self.Cmin, self.nC, self.pf, self.k0, self.k1, self.k2, self.R, self.Dt  = Cmax, Cmin, nC, pf, k0, k1, k2, R, Dt
        """
        Define the Model parameters
        """
        
        self.p = np.ones(self.nC) # transition probability 
        self.populate_p() # ppupalate with the probabilities
        self.q = 1 - self.p
        self.C = np.linspace(self.Cmax,self.Cmin,self.nC)
        self.beta = np.exp(-self.R*self.Dt)
        self.T = (self.nC-1)*3 + 1
        self.nESS = self.nC * (self.nC + 1) * (2*self.nC+1)/6

        self.firm1 = np.empty((self.nC,self.nC,self.nC)) # initialize the two firms with empty values 
        self.firm1[:] = np.nan
        self.firm2 = self.firm1.copy()
        self.populate_firms()
        
        self.stage_index = self.rlsindex(self.nC)

        # Functional forms
        """
        Lambda functions: xxx (Describe these...)
        K
        Payoff
        r1
        r2
        Phi
        """
        self.K = lambda c: self.k0+self.k1/(1+self.k2*c) # investment cost of state of the art production cost c
        self.payoff = lambda x: (self.p-x)*self.Dt  # flow payoffs per unit of time
        self.r1 = lambda x1,x2: np.maximum(x2-x1,0)
        self.r2 = lambda x1,x2: np.maximum(x1-x2,0)
        self.Phi = lambda vN,vI: np.maximum(vN, vI) # xxx this should maybe be made to accept lists.

    """
    xxx this is a very long if function :(
    to do:
    fix the functions dependent on mp (Think it's the same as self)
    fix all the for loops, might have to start on 0
    define all the self.solve functions
    and define the functions:
        ESS
            bases
            index
        ss
            nEQ
            EQs
                eq
        EQ
        
    """
    def state_recursion(self,ss,ESS,tau,mp): 
        if tau == self.T:
            ss,Ess = self.solve_last_corner(ss,ESS,mp)
            tau -= 1
        if tau == self.T -1:
            ss,Ess = self.solve_last_edge(ss,ESS,mp)
            tau -= 1
        if tau == self.T -2:
            ss,Ess = self.solve_last_interior(ss,ESS,mp)
            tau -= 1
        
        dothis = 1
        while dothis == 1: # break when tau=0 
            if np.remainder(tau,3)==1: 
                ic = np.ceil((tau+2)/3)
                ss,ESS = self.solve_corner(ss,ic,ESS,mp)
                tau -= 1
                if tau == 0:
                    break
            if np.remainder(tau,3)==0:
                ic = np.ceil((tau+2)/3)
                ss, ESS = self.solve_edge(ss,ic,ESS,mp)
                tau -= 1
            if np.remainder(tau,3) == 2:
                ic = ceil((tau+2)/3)
                ss, ESS = self.solve_interior(ss,ic,ESS,mp)
                tau -= 1
        # end of the scary while loop
        return ss, ESS
    
    # define all the solve functions used by state_recursion
    def solve_last_corner(self,ss,ESS,mp):
        h = self.nC # Number of technological levels
        c = self.Cmin # State of the art marginal cost for last tech. level

        # Both players have state of the art technology implemented ic1=ic2=c

        # If K>0 the vN1 = r1/(1-beta) .... geometric sum
        vN1 = (self.r1(c,c)+self.beta * np.maximum(0,-self.K(c)))  /  (1-self.beta)
        vI1 = vN1 - self.K(c)
        P1 = vI1 > vN1 # Equivalent to 0>self.K(c)
        vN2 = (self.r2(c,c)+self.beta * np.maximum(0 , -self.K(c)))  /  (1-self.beta)
        vI2 = vN2 - self.K(c)
        P2 = vI2 > vN2 # Equivalent to 0>self.K(c) and hence equal to P1
        
        # OUTPUT is stored in ss
        ss(h).EQs(h,h,1).eq = self.EQ(P1,vN1,vI1,P2,vN2,vI2)
        # Only one equilibrium is possible:
        ss(h).nEQ(h,h) = 1
        ESS.bases(ESS.index(h,h,h)) = 1
        return ss, ESS

    """
    xxx
    To do:
    Define all of the functions used :(
    """
    def solve_last_edge(self,ss,ESS,mp):
        # % INPUT:
        # % cost and mp are global parameters
        # % ss state space structure (has info on eq's in corner of last layer)
        # % OUTPUT:
        # % Equilibria lf.EQ(P1, vN1, vI1, P2, vN2, vI2) for edge state space points
        # % of the final layer:
        # % Final layer <=> s=(x1,x2,c) with c = min(mp.C) 
        # % Edge <=> s=(x1,x2,c) with x2 = c = min(mp.C) and x1 > c or
        # % s=(x1,x2,c) with x1 = c = min(mp.C) and x2 > c

        ic = self.nC # Get the level of technology final layer
        c = self.Cmin # Get state of the art marginal cost for tech. of final layer

        h = 1
        # % h is used to select equilibria in the corner of the final layer but there
        # % is only ever 1 equilibria in the corner
        # % If we did not apply this apriori knowledge we would have to use ESS
        # % max(vN,vI | at the corner final layer)= mp.Phi(ss(ic).EQs(ic,ic,h).eq.vN1,ss(ic).EQs(ic,ic,h).eq.vI1)

        # Get the value of max choice in the corner of final layer s = (c,c,c)
        # xxx lots of nested functions here that are not defined
        g1_ccc = np.maximum(ss(ic).EQs(ic,ic,h).eq.vN1,ss(ic).EQs(ic,ic,h).eq.vI1)
        g2_ccc = np.maximum(ss(ic).EQs(ic,ic,h).eq.vN2,ss(ic).EQs(ic,ic,h).eq.vI2)

        # Player 2 is at the edge s=(x1,x2,c) with x2=c=min(mp.C) and x1>c
        # xxx start on 0?
        for ic1 in range(1,ic-1):
            x1 = self.C(ic1)
            vI1 = self.r1(x1,c) - self.K(c)  + self.beta * g1_ccc
            vN1search = lambda z: self.r1(x1,c) + self.beta * self.Phi(z,vI1) - z
            vN1 = optimize.fsolve(vN1search,0)
            P1 = vI1 > vN1


            vN2 = ( self.r2(x1,c) + self.beta * (P1*g2_ccc+(1-P1)*self.Phi(0,-mp.K(c))) )  /  ( 1-self.beta*(1-P1) )
            vI2 = vN2 - self.K(c)
            P2 = vI2 > vN2

            ss(ic).EQs(ic1,ic,h).eq = lf.EQ(P1, vN1, vI1, P2, vN2 , vI2)
            ss(ic).nEQ(ic1,ic) = 1
            ESS.bases(ESS.index(ic1,ic,ic)) = 1
        
        # xxx maybe start at 0?
        # Player 1 is at the edge s=(x1,x2,c) with x1=c=min(mp.C) and x2>c
        for ic2 in range(1,ic-1):
            x2 = self.C(ic2)
            vI2 = self.r2(c,x2) - self.K(c) + self.beta * g2_ccc
            vN2search = lambda x: self.r2(c, x2) + self.beta*self.Phi(x,vI2)-x
            vN2 = optimize.fsolve(vN2search,0)
            P2 = vI2 > vN2


            vN1 = (self.r1(c, x2) + self.beta*(P2*g1_ccc+(1-P2)*self.Phi(0, -self.K(c))))  /  ( 1-self.beta*(1-P2) )
            vI1 = vN1-self.K(c)
            P1 = vI1 > vN1

            ss(ic).EQs(ic, ic2, 1).eq = self.EQ(P1, vN1, vI1, P2, vN2, vI2)
            ss(ic).nEQ(ic, ic2) = 1
            ESS.bases(ESS.index(ic,ic2,ic)) = 1

        return ss,ESS
    """
    To do:
    Define the functions used :(
    """
    def solve_last_interior(self,ss,ESS,mp):
        # outside loop
        ic = self.nC
        c = self.C(ic)

        """
        These very long lambda functions have a lot of stuff that's not defined
        I think they just find whether investing or not investing gives highest utility
        """
        g1 = lambda iC1, iC2, iC: np.maximum(ss(iC).EQs(iC1, iC2, ESS.esr(ESS.index(iC1,iC2,iC))+1).eq.vN1,ss(iC).EQs(iC1, iC2, ESS.esr(ESS.index(iC1,iC2,iC))+1).eq.vI1)
        g2 = lambda iC1, iC2, iC: np.maximum(ss(iC).EQs(iC1, iC2, ESS.esr(ESS.index(iC1,iC2,iC))+1).eq.vN2,ss(iC).EQs(iC1, iC2, ESS.esr(ESS.index(iC1,iC2,iC))+1).eq.vI2)

    # xxx start on 0?
    for ic1 in range(1,ic-1): #Player 1 loop begin
        for ic2 in range(1,ic-1): #Player 2 loop begin                
            # Player 1 -> leads to P2 candidates
            a = mp.r1(mp.C(ic1), mp.C(ic2)) - mp.K(c) + mp.beta*g1(ic, ic2, ic); %check
            b = mp.beta*(g1(ic, ic, ic)-g1(ic, ic2, ic)); % check
            d = mp.r1(mp.C(ic1),mp.C(ic2));
            e = mp.beta*g1(ic1, ic, ic);


            b_0 = - mp.beta * b; % check 
            b_1 = mp.beta * g1(ic1, ic, ic) + (mp.beta-1)*b - mp.beta*a; % check
            b_2 = mp.r1(mp.C(ic1),mp.C(ic2)) + (mp.beta-1) * a; % check 


            pstar2 = lf.quad(b_0, b_1, b_2); 
            % always return 1 and 0 for the pure strategies


            % Player 2 -> leads to P1 candidates
            A = mp.r2(mp.C(ic1), mp.C(ic2)) - mp.K(c) + mp.beta*g2(ic1, ic, ic); 
            B = mp.beta*(g2(ic, ic, ic)-g2(ic1, ic, ic));
            D = mp.r2(mp.C(ic1),mp.C(ic2));
            E = mp.beta*g2(ic, ic2, ic);

            d_0 = - mp.beta * B;
            d_1 = mp.beta*g2(ic, ic2, ic) + (mp.beta-1) * B - mp.beta*A;
            d_2 = mp.r2(mp.C(ic1),mp.C(ic2)) + (mp.beta-1) * A;
            
            pstar1 = lf.quad(d_0, d_1, d_2);
                
                % Find equilibria based on candidates
                % Number of equilibria found are 0 to begin with
            count = 0;
            for i = 1:length(pstar1)
                    for j = 1:length(pstar2)
                    if all(ismember([i,j],[1,2])) % these are pure strategies
                    % If the polynomial is negative vI > vN
                    % hence player invests set exPj=1 else 0
                    % exP1 is best response to pstar2(j)
                    exP1 = b_2 + b_1 * pstar2(j) + b_0 * pstar2(j)^2 < 0 ;
                    exP2 = d_2 + d_1 * pstar1(i) + d_0 * pstar1(i)^2 < 0 ;

                    % check if both are playing best response
                    % in pure strategies. Players best response
                    % should be equal to the candidate to which
                    % the other player is best responding.
                    if abs(exP1 - pstar1(i)) < 1e-8 && abs(exP2-pstar2(j)) < 1e-8;
                    % if exP1=0 and pstar_i=0 true
                    % if exP1=1 and pstar_i=1 true
                    % Testing whether best response exP1 is
                    % equal to pstar1(i) to which Player 2
                    % is best responding ...
                    count = count + 1;
                    vI1 = a + b*pstar2(j); 
                    vN1 = (d + e*pstar2(j) + mp.beta*(1-pstar2(j))*(a+b*pstar2(j)))*pstar1(i)     +     (1-pstar1(i))*(d+e*pstar2(j))/(1-mp.beta*(1-pstar2(j)));
                    vI2 = A + B*pstar1(i); 
                    vN2 = (D + E*pstar1(i) + mp.beta*(1-pstar1(i))*(A+B*pstar1(i)))*pstar2(j)     +     (1-pstar2(j))*(D+E*pstar1(i))/(1-mp.beta*(1-pstar1(i)));

                        ss(ic).EQs(ic1, ic2, count).eq = lf.EQ(pstar1(i),vN1,vI1,pstar2(j),vN2,vI2);
                    end
                    elseif i > 2 && j > 2 && pstar1(i) >= 0 && pstar2(j) >= 0 && pstar1(i) <= 1 && pstar2(j) <= 1
                        count = count + 1;
                        v1 = a + b * pstar2(j);
                        v2 = A + B * pstar1(i);
                        ss(ic).EQs(ic1, ic2, count).eq = lf.EQ(pstar1(i),v1,v1,pstar2(j),v2,v2);
                    end % end if
                    end % pstar2 loop
        end % pstar1 loop
            ss(ic).nEQ(ic1, ic2) = count; 
            ESS.bases(ESS.index(ic1,ic2,ic)) = count;
    end %Player 2 loop end
    end %Player 1 loop end
    end % end solve_last_interior

    def rlsindex(self, nC):
        Ns = 0
        for i in range(0,nC+1):
            Ns = Ns + i**2
        T = 1 + 3 * (nC-1)
        Ns_in_stages = np.ones((1,T))
        j = 0
        l = nC
        while l>1:
            Ns_in_stages[0,j] = 1
            j += 1
            Ns_in_stages[0,j] = 2*(l-1)
            j += 1
            Ns_in_stages[0,j] = (l-1)**2
            j += 1
            l -= 1
        return np.cumsum(Ns_in_stages)

    def populate_p(self):
        """
        Simple list with 0 as last value
        """

        for a in range(0,self.nC):
            if a < self.nC - 1:
                self.p[a] = self.p[a]*self.pf
            else:
                self.p[a] = 0

    def populate_firms(self):
        for i in range(0,self.nC):
            self.firm2[i,:i+1,:i+1] = np.matlib.repmat(C[:i+1],i+1,1)
            self.firm1[i] = self.firm2[i].transpose().copy()
        

In [550]:
testlf = lf()

4
0
10


In [568]:
P2 = 10 > 20
print(P2)

False


In [516]:
test[0,0]

1.0